'corporate_id': 57

'company': 137

'area': 164

In [5]:
import json
import requests
import pandas as pd

In [12]:
payload = json.dumps(
    {
        "password": "A123456*",
        "email": "bsocompi@cittca.com.bo",
    }
)
headers = {
    'Content-Type': 'application/json'
}
response = requests.request(
    method="POST", 
    url="https://beta-api.naxsolutions.com/api/v1/token/pair", 
    headers=headers, 
    data=payload
)
token = response.json()['access']
token

'eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJ0b2tlbl90eXBlIjoiYWNjZXNzIiwiZXhwIjoxNzU3MzU4ODI3LCJpYXQiOjE3NTcyNzI0MjcsImp0aSI6IjcwYjE4MjI3ZTFhMTQxYmFiYTBhODFlYTU3MTg5ODZjIiwidXNlcl9pZCI6MTU2MSwicm9sZSI6ImFkbWluIiwicGVybWlzc2lvbnMiOlsibm90ZS5jcmVhdGUiLCJub3RlLnJlYWQiLCJub3RlLnVwZGF0ZSIsInZhcmlhYmxlX21hcC5jcmVhdGUiLCJ2YXJpYWJsZV9tYXAucmVhZCIsInZhcmlhYmxlX21hcC51cGRhdGUiLCJkYXNoYm9hcmQuY3JlYXRlIiwiZGFzaGJvYXJkLnJlYWQiLCJkYXNoYm9hcmQudXBkYXRlIiwidGFza19sYWJlbC5jcmVhdGUiLCJ0YXNrX2xhYmVsLnJlYWQiLCJ0YXNrX2xhYmVsLnVwZGF0ZSIsImFyZWEuY3JlYXRlIiwiYXJlYS5yZWFkIiwiYXJlYS51cGRhdGUiLCJ0aWxlLmNyZWF0ZSIsInRpbGUucmVhZCIsInRpbGUudXBkYXRlIiwiY29ycG9yYXRlLmNyZWF0ZSIsImNvcnBvcmF0ZS5yZWFkIiwiY29ycG9yYXRlLnVwZGF0ZSIsImxhYm9yYXRvcnkuY3JlYXRlIiwibGFib3JhdG9yeS5yZWFkIiwibGFib3JhdG9yeS51cGRhdGUiLCJjbGltYXRlX3N0YXRpb24uY3JlYXRlIiwiY2xpbWF0ZV9zdGF0aW9uLnJlYWQiLCJjbGltYXRlX3N0YXRpb24udXBkYXRlIiwiY2xpbWF0ZV9zdGF0aW9uX3JlY29yZC5jcmVhdGUiLCJjbGltYXRlX3N0YXRpb25fcmVjb3JkLnJlYWQiLCJjbGltYXRlX3N0YXRpb25fcmVjb3JkLnVwZGF0ZSIsIm

In [19]:
def get_samplings(token: str, campaign_id: int, board_id: int = None,
                  sort: str = "created_at", direction: str = "ASC",
                  limit: int = 100, offset: int = 0, extra_filters: dict = None):
    
    url = "https://beta-api.naxsolutions.com/api/v1/smart_samplings/samplings"
    headers = {
        "Authorization": f"Bearer {token}"
    }

    params = {
        "campaign_id": campaign_id,
        "sort": sort,
        "direction": direction,
        "limit": limit,
        "offset": offset
    }

    if board_id:
        params["board_id"] = board_id
    if extra_filters:
        params.update(extra_filters)

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        return response.json().get("items", [])
    else:
        print(f"Error {response.status_code}: {response.text}")
        return []

In [20]:
campaign_id = 1111
board_id = 46
samplings = get_samplings(token, campaign_id, board_id)

In [21]:
df_samplings = pd.DataFrame(samplings)
df_samplings = df_samplings[df_samplings['status'] != 'finished']
df_samplings['marker_count'].sum()

145

In [22]:
len(df_samplings)

51

In [23]:
def get_sampling_by_id(token: str, sampling_id: int):
    url = f"https://beta-api.naxsolutions.com/api/v1/smart_samplings/samplings/{sampling_id}"
    headers = {
        "Authorization": f"Bearer {token}"
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error {response.status_code}: {response.text}")
        return None

In [24]:
def parse_sampling_markers(sampling_response):
    sampling_id = sampling_response["id"]
    markers = sampling_response.get("markers", [])

    # Crear una lista de diccionarios con los campos que querés
    rows = []
    for m in markers:
        row = {
            "sampling_id": sampling_id,
            "marker_id": m.get("id"),
            "marker_name": m.get("name"),
            "value": m.get("value"),
            "lot_id": m.get("lot_id"),
            "latitude": m.get("latitude"),
            "longitude": m.get("longitude"),
            "created_at": m.get("created_at"),
            "updated_at": m.get("updated_at"),
        }
        rows.append(row)

    # Convertir en DataFrame
    return pd.DataFrame(rows)

In [25]:
df_res = None
for i, row in df_samplings.iterrows():
    print(row['id'])
    sampling_details = get_sampling_by_id(token, row['id'])
    df_markers = parse_sampling_markers(sampling_details)
    df_res = pd.concat([df_res, df_markers], axis=0, ignore_index=True)

988
989
990
991
992
993
994
995
996
997
998
999
1000
1001
1002
1003
1004
1005
1006
1007
1008
1009
1010
1011
1012
1013
1014
1015
1016
1017
1018
1019
1020
1021
1022
1023
1024
1025
1026
1027
1028
1029
1030
1031
1032
1072
1073
1093
1094
1095
1096


In [26]:
df_res

,sampling_id,marker_id,marker_name,value,lot_id,latitude,longitude,created_at,updated_at
0,988,210178,1_SAUSALITO-CITTCA8522_00001140000L5.14,0.678398,1849950,-17.223590,-63.367489,2025-04-12T20:55:11.875Z,2025-04-12T20:55:11.875Z
1,988,210179,2_SAUSALITO-CITTCA8522_00001140000L5.12,0.836963,1850236,-17.221736,-63.374198,2025-04-12T20:55:11.875Z,2025-04-12T20:55:11.875Z
2,988,210180,3_SAUSALITO-CITTCA8522_00001140000L5.12,0.922213,1850236,-17.220251,-63.373123,2025-04-12T20:55:11.875Z,2025-04-12T20:55:11.875Z
3,988,210181,4_SAUSALITO-CITTCA8522_00001140000L5.9,0.968073,1850110,-17.218495,-63.367343,2025-04-12T20:55:11.875Z,2025-04-12T20:55:11.875Z
4,989,210182,1_SAUSALITO-UCG9020_00001140000L3,0.723312,1845627,-17.228845,-63.369960,2025-04-12T21:01:50.313Z,2025-04-12T21:01:50.313Z
...,...,...,...,...,...,...,...,...,...
140,1096,210859,2_EL CARMEN CIDRAL-UCG9020_00001840000L24,0.984708,1847468,-17.368518,-63.304422,2025-04-20T22:34:38.547Z,2025-04-20T22:34:38.547Z
141,1096,210860,3_EL CARMEN CIDRAL-UCG9020_00001840000L24,0.984708,1847468,-17.368743,-63.305722,2025-04-20T22:34:38.547Z,2025-04-20T22:34:38.547Z
142,1096,210861,4_EL CARMEN CIDRAL-UCG9020_00001840000L30,0.985960,1850472,-17.367095,-63.309939,2025-04-20T22:34:38.547Z,2025-04-20T22:34:38.547Z
143,1096,210862,5_EL CARMEN CIDRAL-UCG9020_00001840000L35,0.985960,1843438,-17.368405,-63.309821,2025-04-20T22:34:38.547Z,2025-04-20T22:34:38.547Z


In [27]:
import geopandas as gpd
from shapely.geometry import Point

In [28]:
# Crear geometría de puntos desde lat/lon
geometry = [Point(xy) for xy in zip(df_res["longitude"], df_res["latitude"])]

# Crear GeoDataFrame con sistema de coordenadas WGS84
gdf = gpd.GeoDataFrame(df_res, geometry=geometry, crs="EPSG:4326")

In [29]:
gdf_utm = gdf.to_crs(epsg=32720)

In [30]:
output_path = "muestreos.shp"
gdf_utm.to_file(output_path, driver='ESRI Shapefile')

C:\Users\bismarksr\AppData\Local\Temp\ipykernel_31288\992117271.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_utm.to_file(output_path, driver='ESRI Shapefile')
